# With Fitness

In [ ]:
import os
os.chdir("..")

In [ ]:
import h5py
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sb
from scipy.stats import erlang, expon, gamma
from scipy.integrate import quad, dblquad, tplquad
from TravelAndMutate.quickanalysis import computeDepths, computeChildren
import TravelAndMutate.datamanager as datman

Check [this link](https://hackmd.io/@dariobaron/SkCdQAdMR) for info and formalism

In [ ]:
file = h5py.File("data/fitness.h5")

params = datman.getHeteroAttributes(file)
empty = []
data = {}
for i,(name,group) in enumerate(file.items()):

	print(f"{i*100//len(params.index)}% - loading infections", end="\r", flush=True)
	simulation = datman.getNameSurvivedSims(group, 1000)[0]
	haplo_infections = group[f"{simulation}/infections"].fields(["mut","loc"])[:]
	print(f"{i*100//len(params.index)}% - computing infections", end="\r", flush=True)
	infectious_haplo = pd.DataFrame.from_records(haplo_infections).groupby("mut").count()
	mostinfectious_haplo = (infectious_haplo.max() / infectious_haplo.sum())["loc"]

	print(f"{i*100//len(params.index)}% - loading mutation tree", end="\r", flush=True)
	mut_tree = group[f"{simulation}/mutationtree"][:]
	print(f"{i*100//len(params.index)}% - computing depths\t\t", end="\r", flush=True)
	depths = computeDepths(mut_tree)
	avg_depth = depths["depth"].mean()
	print(f"{i*100//len(params.index)}% - computing children\t", end="\r", flush=True)
	children = computeChildren(mut_tree)
	best_parent = children["children"].max() / children["children"].sum()

	print(f"{i*100//len(params.index)}% - filtering", end="\r", flush=True)
	unique_haplos = np.unique(haplo_infections["mut"])
	filtered_depths = pd.DataFrame.from_records(depths, index="id").loc[unique_haplos]
	avg_depth_fil = filtered_depths.mean()["depth"]
	filtered_children = pd.DataFrame.from_records(children, index="id").loc[unique_haplos]
	best_parent_fil = (filtered_children.max() / filtered_children.sum())["children"]

	print(f"{i*100//len(params.index)}%\t\t\t\t\t\t\t\t", end="\r", flush=True)
	data[name] = [mostinfectious_haplo, avg_depth, best_parent, avg_depth_fil, best_parent_fil]

data = pd.DataFrame.from_dict(data, orient="index", columns=["mostinfhaplo","avgdepth","bestparent","FILavgdepth","FILbestparent"])

file.close()

In [ ]:
df = pd.concat([params,data], axis=1)
df.sort_index(inplace=True)
df

In [ ]:
betafixed = {beta:group.drop(columns="betas") for beta,group in df.groupby("betas")}

# With $R_0 = 1.1$

In [ ]:
labels = [
	"Fraction of cases by the most infectious haplotypes",
	"Average depth in mutation tree",
	"Fraction of haplotypes generated by the most prolific one"
]
mycolors = ["Reds","Greens","Blues"]
tags = ["mostinfhaplo","avgdepth","bestparent"]
nrows = len(labels)
fig, axs = plt.subplots(nrows, 1, figsize=(8,nrows*4))
for i in range(len(labels)):
	plotting = betafixed[0.022].pivot(index="mutation_rate", columns="mutation_k", values=tags[i])
	if i == 1:
		sb.heatmap(plotting, annot=True, fmt=".2e", cmap=mycolors[i], norm=matplotlib.colors.LogNorm(), ax=axs[i])
	else:
		sb.heatmap(plotting, annot=True, fmt=".2e", cmap=mycolors[i], ax=axs[i])
	axs[i].set_title(labels[i])
	axs[i].invert_xaxis()
	axs[i].set_xlabel("k")
	axs[i].invert_yaxis()
	axs[i].set_ylabel("Avg mutation rate")
fig.suptitle(r"$R_0 = 1.1$", fontsize=20)
fig.tight_layout()
fig.savefig("analysis/fit_R0_1.1.png")

In [ ]:
labels = [
	"Only infectious - Average depth in mutation tree",
	"Only infectious - Fraction of haplotypes generated by the most prolific one"
]
mycolors = ["Greens","Blues"]
tags = ["FILavgdepth","FILbestparent"]
nrows = len(labels)
fig, axs = plt.subplots(nrows, 1, figsize=(8,nrows*4))
for i in range(len(labels)):
	plotting = betafixed[0.022].pivot(index="mutation_rate", columns="mutation_k", values=tags[i])
	if i == 0:
		sb.heatmap(plotting, annot=True, fmt=".2e", cmap=mycolors[i], norm=matplotlib.colors.LogNorm(), ax=axs[i])
	else:
		sb.heatmap(plotting, annot=True, fmt=".2e", cmap=mycolors[i], ax=axs[i])
	axs[i].set_title(labels[i])
	axs[i].invert_xaxis()
	axs[i].set_xlabel("k")
	axs[i].invert_yaxis()
	axs[i].set_ylabel("Avg mutation rate")
fig.suptitle(r"$R_0 = 1.1$", fontsize=20)
fig.tight_layout()
fig.savefig("analysis/fit_R0_1.1-reduced.png")

# With $R_0 = 1.5$

In [ ]:
labels = [
	"Fraction of cases by the most infectious haplotypes",
	"Average depth in mutation tree",
	"Fraction of haplotypes generated by the most prolific one"
]
mycolors = ["Reds","Greens","Blues"]
tags = ["mostinfhaplo","avgdepth","bestparent"]
nrows = len(labels)
fig, axs = plt.subplots(nrows, 1, figsize=(8,nrows*4))
for i in range(len(labels)):
	plotting = betafixed[0.03].pivot(index="mutation_rate", columns="mutation_k", values=tags[i])
	if i == 1:
		sb.heatmap(plotting, annot=True, fmt=".2e", cmap=mycolors[i], norm=matplotlib.colors.LogNorm(), ax=axs[i])
	else:
		sb.heatmap(plotting, annot=True, fmt=".2e", cmap=mycolors[i], ax=axs[i])
	axs[i].set_title(labels[i])
	axs[i].invert_xaxis()
	axs[i].set_xlabel("k")
	axs[i].invert_yaxis()
	axs[i].set_ylabel("Avg mutation rate")
fig.suptitle(r"$R_0 = 1.5$", fontsize=20)
fig.tight_layout()
fig.savefig("analysis/fit_R0_1.5.png")

In [ ]:
labels = [
	"Only infectious - Average depth in mutation tree",
	"Only infectious - Fraction of haplotypes generated by the most prolific one"
]
mycolors = ["Greens","Blues"]
tags = ["FILavgdepth","FILbestparent"]
nrows = len(labels)
fig, axs = plt.subplots(nrows, 1, figsize=(8,nrows*4))
for i in range(len(labels)):
	plotting = betafixed[0.03].pivot(index="mutation_rate", columns="mutation_k", values=tags[i])
	if i == 0:
		sb.heatmap(plotting, annot=True, fmt=".2e", cmap=mycolors[i], norm=matplotlib.colors.LogNorm(), ax=axs[i])
	else:
		sb.heatmap(plotting, annot=True, fmt=".2e", cmap=mycolors[i], ax=axs[i])
	axs[i].set_title(labels[i])
	axs[i].invert_xaxis()
	axs[i].set_xlabel("k")
	axs[i].invert_yaxis()
	axs[i].set_ylabel("Avg mutation rate")
fig.suptitle(r"$R_0 = 1.5$", fontsize=20)
fig.tight_layout()
fig.savefig("analysis/fit_R0_1.5-reduced.png")

# With $R_0 = 2.5$

In [ ]:
labels = [
	"Fraction of cases by the most infectious haplotypes",
	"Average depth in mutation tree",
	"Fraction of haplotypes generated by the most prolific one"
]
mycolors = ["Reds","Greens","Blues"]
tags = ["mostinfhaplo","avgdepth","bestparent"]
nrows = len(labels)
fig, axs = plt.subplots(nrows, 1, figsize=(8,nrows*4))
for i in range(len(labels)):
	plotting = betafixed[0.05].pivot(index="mutation_rate", columns="mutation_k", values=tags[i])
	if i == 1:
		sb.heatmap(plotting, annot=True, fmt=".2e", cmap=mycolors[i], norm=matplotlib.colors.LogNorm(), ax=axs[i])
	else:
		sb.heatmap(plotting, annot=True, fmt=".2e", cmap=mycolors[i], ax=axs[i])
	axs[i].set_title(labels[i])
	axs[i].invert_xaxis()
	axs[i].set_xlabel("k")
	axs[i].invert_yaxis()
	axs[i].set_ylabel("Avg mutation rate")
fig.suptitle(r"$R_0 = 2.5$", fontsize=20)
fig.tight_layout()
fig.savefig("analysis/fit_R0_2.5.png")

In [ ]:
labels = [
	"Only infectious - Average depth in mutation tree",
	"Only infectious - Fraction of haplotypes generated by the most prolific one"
]
mycolors = ["Greens","Blues"]
tags = ["FILavgdepth","FILbestparent"]
nrows = len(labels)
fig, axs = plt.subplots(nrows, 1, figsize=(8,nrows*4))
for i in range(len(labels)):
	plotting = betafixed[0.05].pivot(index="mutation_rate", columns="mutation_k", values=tags[i])
	if i == 0:
		sb.heatmap(plotting, annot=True, fmt=".2e", cmap=mycolors[i], norm=matplotlib.colors.LogNorm(), ax=axs[i])
	else:
		sb.heatmap(plotting, annot=True, fmt=".2e", cmap=mycolors[i], ax=axs[i])
	axs[i].set_title(labels[i])
	axs[i].invert_xaxis()
	axs[i].set_xlabel("k")
	axs[i].invert_yaxis()
	axs[i].set_ylabel("Avg mutation rate")
fig.suptitle(r"$R_0 = 2.5$", fontsize=20)
fig.tight_layout()
fig.savefig("analysis/fit_R0_2.5-reduced.png")

# With $R_0 = 3.5$

In [ ]:
labels = [
	"Fraction of cases by the most infectious haplotypes",
	"Average depth in mutation tree",
	"Fraction of haplotypes generated by the most prolific one"
]
mycolors = ["Reds","Greens","Blues"]
tags = ["mostinfhaplo","avgdepth","bestparent"]
nrows = len(labels)
fig, axs = plt.subplots(nrows, 1, figsize=(8,nrows*4))
for i in range(len(labels)):
	plotting = betafixed[0.07].pivot(index="mutation_rate", columns="mutation_k", values=tags[i])
	if i == 1:
		sb.heatmap(plotting, annot=True, fmt=".2e", cmap=mycolors[i], norm=matplotlib.colors.LogNorm(), ax=axs[i])
	else:
		sb.heatmap(plotting, annot=True, fmt=".2e", cmap=mycolors[i], ax=axs[i])
	axs[i].set_title(labels[i])
	axs[i].invert_xaxis()
	axs[i].set_xlabel("k")
	axs[i].invert_yaxis()
	axs[i].set_ylabel("Avg mutation rate")
fig.suptitle(r"$R_0 = 3.5$", fontsize=20)
fig.tight_layout()
fig.savefig("analysis/fit_R0_3.5.png")

In [ ]:
labels = [
	"Only infectious - Average depth in mutation tree",
	"Only infectious - Fraction of haplotypes generated by the most prolific one"
]
mycolors = ["Greens","Blues"]
tags = ["FILavgdepth","FILbestparent"]
nrows = len(labels)
fig, axs = plt.subplots(nrows, 1, figsize=(8,nrows*4))
for i in range(len(labels)):
	plotting = betafixed[0.07].pivot(index="mutation_rate", columns="mutation_k", values=tags[i])
	if i == 0:
		sb.heatmap(plotting, annot=True, fmt=".2e", cmap=mycolors[i], norm=matplotlib.colors.LogNorm(), ax=axs[i])
	else:
		sb.heatmap(plotting, annot=True, fmt=".2e", cmap=mycolors[i], ax=axs[i])
	axs[i].set_title(labels[i])
	axs[i].invert_xaxis()
	axs[i].set_xlabel("k")
	axs[i].invert_yaxis()
	axs[i].set_ylabel("Avg mutation rate")
fig.suptitle(r"$R_0 = 3.5$", fontsize=20)
fig.tight_layout()
fig.savefig("analysis/fit_R0_3.5-reduced.png")